# Player Analysis for BPL 10

Author: Cearlmau

This file breaks down the player distribution of BPL teams based on team and level
at various points in the BPL 10 event. The goal of this data analysis is to look at
whether or not the team distributions had an effect on overall team player levels, and
how that may contribute to a team's success.

Special thanks to Moowiz for providing the data.

In [35]:
#Libraries and Data Sources
import csv
import pandas as pd
import plotly
import datetime

level_data = pd.read_csv("../../data/bpl-10_level_data.csv")
teams_data = pd.read_csv("../../data/bpl-10_teams.csv")

## Background:

Path of Exile is a popular game whose premise revolves around the player killing enemies using a preset character with an extensive amount of customization. The killed enemies generate loot and other bonuses which the player can use to augment and upgrade their character. Path of Exile is an online game, which means that players can interact with one another, trading each items and/or playing in the same instance as one another.

BPL is an quarterly event that coincides with each major update that Path of Exile receives. Around 2000 players is placed inside an isolated and newly created instance (referred to as a "league") where they must work together with their designated team to acquire items and accomplish objectives("bounties") defined by the organizers. Doing so accumulates points towards their team total, and at the end the teams are ranked based on how many points they have acquired over the four-day event. There are 4 teams for BPL 10.

Each preset character has a specific upgrade called an "ascendancy". This allows the character to gain unique powers that are inaccesible to other presets. Each character is permitted only one ascendancy. To balance out the teams, the organizers have made it so that each team can only have a subset of the presets. One of the ascendancies, "Sabateur", is banned.

One thing to note is that ascendancy bonuses are acquired in portions. To be fully ascended is to grab all possible upgrades related to that ascendancy.

Each character also has a stage-based development referred to as "levels". Characters will start off weak. As they kill enemies they acquire numerical reward ("experience points"). When they have acquired sufficient experience points, they will "level up", and achieve the next stage of their character progression. In Path of Exile, a character can have levels from 1 to 100, inclusive of both ends. Typically, having more levels mean that your character becomes stronger.

The playthrough of a character is broken down into two parts: campaign, where the character goes through a story-driven progression, and endgame, where the player can tackle various content unavailable in campaign. All of BPL's bounties occur in the endgame. 

A large portion of the endgame takes place in the Atlas, where players can enter special areas to fight harder enemies and acquire more rewarding loot. Completing sections of the Atlas awards players with the power to customize the Atlas in the form of "atlas passives". Note that is it not required that a player needs all atlas passives to participate in the endgame, but having more gives the player more options in their playstyle.

The data is broken down into two csvs. 
**bpl-10_teams.csv** contains information at the end of the event. It contains finalized data of all the players, including level, whether or not a player has fully ascended, and the number of atlas passives they have acquired. 
**bpl-10_level_data.csv** contains progressive tracking of every single character's levelling over the course of the event. Snapshots of each character are taken at approximately every 15 minutes. Each entry contains the player, their team name, their character level, and time of snapshot.

For anonymity and ethics, player names are replaced with hashed values.


In [135]:
'''
The first thing we have to do is clean up the data.
While there are no duplicates or missing values,
the levelling data is inconsistent. Since data collection
happened in real time, the timestamps can differ by seconds,
even within the same snapshot.
'''
level_data.astype({'team': 'string', 'char_name_hash': 'string'})
level_data = level_data.dropna()
bpl_start_time = 1673668800.0
test_row = level_data["time_fetched"][1]
index = 1
def convertToEpoch(test_row):
    format = "%Y-%m-%d %H:%M:%S"
    tm = datetime.datetime.strptime(test_row['time_fetched'], format)
    discard = datetime.timedelta(minutes=tm.minute % 15,
                                seconds=tm.second
                                )
    tm -= discard
    if discard >= datetime.timedelta(minutes=7.5):
        tm += datetime.timedelta(minutes=15)
    t =  (tm.timestamp() - bpl_start_time)/(900)
    return t

level_data["time"] = level_data.apply(lambda row: convertToEpoch(row), axis=1)
print(level_data.tail(10))
print(level_data['time'].max())

  team  char_level                    char_name_hash         time_fetched  \
0  MER          36  e771d114910bddeb00e518ce1bbf8afe  2023-01-13 20:12:09   
1  ROY          35  2f36664401b6de490dae3524fb47bdfb  2023-01-13 20:12:09   
2  MER          35  14ad8244e33391d820e84f5ac7ef7684  2023-01-13 20:12:09   
3  MER          35  bd0beba8993740a12353e95560137047  2023-01-13 20:12:09   
4  EXP          35  750d95f47c5ea4490fe92507e6a23ce8  2023-01-13 20:12:09   
5  MER          35  a28b0c4ae3f537cab65a6e02d7a14a31  2023-01-13 20:12:09   
6  ROY          35  953183881aafcf467ccbe0b780cd0f2d  2023-01-13 20:12:09   
7  COR          34  3edf1977e9cbb5e08a6b7a475f34d3d4  2023-01-13 20:12:09   
8  EXP          34  92e6d065bad8376ce24d1e2732e5bcd1  2023-01-13 20:12:09   
9  MER          34  595d2d3fdffe8b408667df50f54ab9a1  2023-01-13 20:12:09   

   time  
0   1.0  
1   1.0  
2   1.0  
3   1.0  
4   1.0  
5   1.0  
6   1.0  
7   1.0  
8   1.0  
9   1.0  
381.0


In [198]:
#This allows us to group by character name so that we have "level at time intervals" as columns
level_data_grouped = level_data.drop(['time_fetched'], axis=1).drop_duplicates(keep='first')
n = len(pd.unique(level_data_grouped['time']))
print(n)
level_data_grouped = level_data_grouped.pivot_table( 
                                    index=['char_name_hash','team'],
                                    values='char_level', 
                                    columns='time',
                                    aggfunc='first'
                                    )



375


c:\Users\cooki\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\cooki\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\cooki\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\cooki\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\cooki\AppData\Local\Programs\Python\Python310\lib\site-packages\pandas\core\internals\blocks.py:2323: RuntimeWarning: invalid value encountered in cast
  values = values.astype(str)
c:\Users\cooki\AppData\Local\Programs\Python\

'char_name_hash,team,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,13.0,14.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0,31.0,32.0,33.0,34.0,35.0,36.0,37.0,38.0,39.0,40.0,41.0,42.0,43.0,44.0,45.0,47.0,48.0,49.0,50.0,51.0,52.0,53.0,54.0,55.0,56.0,57.0,58.0,59.0,60.0,61.0,62.0,63.0,64.0,65.0,66.0,67.0,68.0,69.0,70.0,71.0,72.0,73.0,74.0,75.0,76.0,77.0,78.0,79.0,80.0,81.0,82.0,83.0,84.0,85.0,86.0,87.0,88.0,89.0,90.0,91.0,92.0,93.0,94.0,95.0,96.0,97.0,98.0,99.0,100.0,101.0,102.0,103.0,104.0,105.0,108.0,109.0,110.0,111.0,112.0,113.0,114.0,115.0,116.0,117.0,118.0,121.0,122.0,123.0,124.0,125.0,126.0,127.0,128.0,129.0,130.0,131.0,132.0,133.0,134.0,135.0,136.0,137.0,138.0,139.0,140.0,141.0,142.0,143.0,144.0,145.0,146.0,147.0,148.0,149.0,150.0,151.0,152.0,153.0,154.0,155.0,156.0,157.0,158.0,159.0,160.0,161.0,162.0,163.0,164.0,165.0,166.0,167.0,168.0,169.0,170.0,171.0,172.0,173.0,174.0,175.0,176.0,177.0,178.0,179.0,180.0,181.0,182.0,183.0,184.0,185.0,186.0,187.0